# RAG LLM 

### The Setup

In [17]:
import os
from dotenv import load_dotenv
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from operator import itemgetter
import tempfile
import whisper
from pytube import YouTube # if you want to do videos in your plug and play 
import pypdf
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_pinecone import PineconeVectorStore

### Credentials

In [5]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

### Model

In [11]:
model = ChatOpenAI(api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")
parser = StrOutputParser()
chain = model | parser
chain.invoke("hi, are you there gpt?")

"Yes, I'm here. How can I assist you today?"

### Plug and Play Context 🚩

In [22]:
file_path = "../files/Computer Age Statistical Inference Book.pdf" # this would be your plug and play pdf file 
with open(file_path, 'rb') as file:
    pdf_reader = pypdf.PdfReader(file)
    
    all_text = []

    for page in pdf_reader.pages:
        all_text.append(page.extract_text())

print(''.join(filter(None, all_text)))

The twenty-first century has seen a 
breathtaking expansion of statistical methodology, both in scope and in influence. “Big data,” “data science,” and “machine learning” have become familiar terms in the news, as statistical methods are brought to bear upon the enormous data sets of modern science and commerce. How did we get here? And where are we going?
This book takes us on an exhilarating 
journey through the revolution in data analysis following the introduction of electronic computation in the 1950s. Beginning with classical inferential theories – Bayesian, frequentist, Fisherian – individual chapters take up a series of influential topics: survival analysis, logistic regression, empirical Bayes, the jackknife and bootstrap, random forests, neural networks, Markov chain Monte Carlo, inference after model selection, and dozens more. The distinctly modern approach integrates methodology and algorithms with statistical inference. The book ends with speculation on the future directi

### Chunking Context for LLM 

In [23]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
text_splitter.split_documents(all_text)


AttributeError: 'str' object has no attribute 'page_content'